In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy as dc

In [2]:
def cvs_to_filtered_dataframe(origin_filename: str):
    # Read data from ./data folder
    data = pd.read_csv(origin_filename)

    # convert to pandas dataframe
    df = pd.DataFrame(data)

    # Specify the day you want to filter for
    # desired_day = '2023-08-01'
    
    # Convert 'time' column to datetime format
    df['time'] = pd.to_datetime(df['ISO Time'])
    
    # Extract date from 'time' column
    # df['Date'] = df['time'].dt.date
    
    # Filtering based on '22102' column and the specified day
    # filtered_df = df[(df['Date'] == pd.to_datetime(desired_day).date()) & (df['22101'] == 1)]
    filtered_df = df
    
    # Drop the temporary 'Date' column if you don't need it in the final result
    # filtered_df = filtered_df.drop(columns=['Date'])
    
    # Extracting time and energy_consumption_per_day columns
    filtered_df = filtered_df[['time','Energy consumed per day (kWh)']]
    
    filtered_df['time'] = pd.to_datetime(filtered_df['time'])

    return filtered_df

In [3]:
def sliding_window_format(df: pd.DataFrame, window_size: int):
    df = dc(df)
    
    df.set_index('time', inplace=True)
    
    for i in range(1, window_size+1):
        df[f'energy(t-{window_size+1-i})'] = df['Energy consumed per day (kWh)'].shift(window_size+1-i)
        
    df.dropna(inplace=True)
    
    return df

In [4]:
def save_dataframe(df: pd.DataFrame, destination_filename: str):
    df.to_csv(destination_filename, index=True)

In [5]:
def retrieve_dataframe(origin_filename: str):
    read_df = pd.read_csv(origin_filename)
    read_df.set_index('time', inplace=True)
    return read_df

In [14]:
for i in range (20):
    df = cvs_to_filtered_dataframe(f'./data/raw/file_{i+1}.csv')
    df = sliding_window_format(df,window_size=7)
    save_dataframe(df, f'./data/formatted/file_{i+1}.csv')

/var/folders/bv/zk0mfpz57lq0x1z9tqfbb7_c0000gp/T/ipykernel_2642/3536648992.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['time'] = pd.to_datetime(filtered_df['time'])
/var/folders/bv/zk0mfpz57lq0x1z9tqfbb7_c0000gp/T/ipykernel_2642/3536648992.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['time'] = pd.to_datetime(filtered_df['time'])
/var/folders/bv/zk0mfpz57lq0x1z9tqfbb7_c0000gp/T/ipykernel_2642/3536648992.py:27: SettingWithCopyWarning: 
A value is trying to be set on a 

In [15]:
frame_list = []

for i in range(20):
    df = retrieve_dataframe(f'./data/formatted/file_{i+1}.csv')
    frame_list.append(df)
    
df = pd.concat(frame_list)
save_dataframe(df,'./data/formatted/final_file.csv')

In [16]:
df = retrieve_dataframe('./data/formatted/final_file.csv')

In [17]:
df

Energy consumed per day (kWh)  energy(t-7)  \
time                                                                    
2023-10-02 06:58:30-06:00                       5.299961     0.204814   
2023-10-02 06:59:00-06:00                       6.206462     0.738663   
2023-10-02 06:59:30-06:00                       6.317779     2.332002   
2023-10-02 07:00:00-06:00                       6.325733     2.714123   
2023-10-02 07:00:30-06:00                       7.320866     3.666794   
...                                                  ...          ...   
2023-10-30 22:12:30-06:00                     402.719293   399.914098   
2023-10-30 22:13:00-06:00                     402.885643   399.961119   
2023-10-30 22:13:30-06:00                     402.896596   400.601987   
2023-10-30 22:49:30-06:00                     402.939250   401.298187   
2023-10-30 22:50:00-06:00                     402.974245   402.049418   

                           energy(t-6)  energy(t-5)  energy(t-4)  energy(t-3)  \
time                                                                            
2023-10-02 06:58:30-06:00     0.738663     2.332002     2.714123     3.666794   
2023-10-02 06:59:00-06:00     2.332002     2.714123     3.666794     4.347329   
2023-10-02 06:59:30-06:00     2.714123     3.666794     4.347329     4.838311   
2023-10-02 07:00:00-06:00     3.666794     4.347329     4.838311     5.299961   
2023-10-02 07:00:30-06:00     4.347329     4.838311     5.299961     6.206462   
...                                ...          ...          ...          ...   
2023-10-30 22:12:30-06:00   399.961119   400.601987   401.298187   402.049418   
2023-10-30 22:13:00-06:00   400.601987   401.298187   402.049418   402.258321   
2023-10-30 22:13:30-06:00   401.298187   402.049418   402.258321   402.597228   
2023-10-30 22:49:30-06:00   402.049418   402.258321   402.597228   402.719293   
2023-10-30 22:50:00-06:00   402.258321   402.597228   402.719293   402.885643   

                           energy(t-2)  energy(t-1)  
time                                                 
2023-10-02 06:58:30-06:00     4.347329     4.838311  
2023-10-02 06:59:00-06:00     4.838311     5.299961  
2023-10-02 06:59:30-06:00     5.299961     6.206462  
2023-10-02 07:00:00-06:00     6.206462     6.317779  
2023-10-02 07:00:30-06:00     6.317779     6.325733  
...                                ...          ...  
2023-10-30 22:12:30-06:00   402.258321   402.597228  
2023-10-30 22:13:00-06:00   402.597228   402.719293  
2023-10-30 22:13:30-06:00   402.719293   402.885643  
2023-10-30 22:49:30-06:00   402.885643   402.896596  
2023-10-30 22:50:00-06:00   402.896596   402.939250  

[29400 rows x 8 columns]